In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

Using cuDNN version 6021 on context None
Mapped name None to device cuda0: GeForce GTX TITAN X (0000:04:00.0)
Using Theano backend.


## Setup

In [2]:
path = "data/dogscats/"
# path = "data/dogscats/sample/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=32
# batch_size=1

In [3]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer
2) Add this to a model containing all VGG layers except the last layer
3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)
4) Add data augmentation, fine-tuning the dense layers without pre-computation.

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [5]:
model = Vgg16().model
conv_layers,fc_layers = split_at(model, Convolution2D)

In [6]:
conv_model = Sequential(conv_layers)

In [7]:
val_features = conv_model.predict_generator(val_batches, int(np.ceil(val_batches.samples/batch_size)))
trn_features = conv_model.predict_generator(batches, int(np.ceil(batches.samples/batch_size)))

In [8]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In the future we can just load these precomputed features:

In [9]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [10]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [11]:
save_array(model_path+'train_data.bc', trn)
save_array(model_path+'valid_data.bc', val)

In the future we can just load these resized images:

In [12]:
trn = load_array(model_path+'train_data.bc')
val = load_array(model_path+'valid_data.bc')

Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [13]:
model.pop()
model.pop()

In [14]:
ll_val_feat = model.predict_generator(val_batches, int(np.ceil(val_batches.samples/batch_size)))
ll_feat = model.predict_generator(batches, int(np.ceil(batches.samples/batch_size)))

In [15]:
save_array(model_path + 'train_ll_feat.bc', ll_feat)
save_array(model_path + 'valid_ll_feat.bc', ll_val_feat)

In [16]:
ll_feat = load_array(model_path+ 'train_ll_feat.bc')
ll_val_feat = load_array(model_path + 'valid_ll_feat.bc')

...and let's also grab the test data, for when we need to submit:

In [17]:
test = get_data(path+'test')
save_array(model_path+'test_data.bc', test)

Found 12500 images belonging to 1 classes.


In [18]:
test = load_array(model_path+'test_data.bc')

## Last layer

The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [19]:
def get_ll_layers():
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [20]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), epochs=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), epochs=1)
    ll_model.save_weights(model_path+'ll_bn' + i + '.h5')

    vgg = Vgg16BN()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(model_path+'bn' + i + '.h5')
    return model

## Dense model

In [21]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]

    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [22]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [23]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, epochs=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    # width_zoom_range removed from the following because not available in Keras2
    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
       channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    steps_per_epoch = int(np.ceil(batches.n/batch_size))
    validation_steps = int(np.ceil(val_batches.n/batch_size))

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    conv_model.save_weights(model_path+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=1, 
                            validation_data=val_batches, validation_steps=validation_steps)
    for layer in conv_model.layers[16:]: layer.trainable = True
        
    #- added again the compile instruction in order to avoid a Keras 2.1 warning message
    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    
    conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=8, 
                            validation_data=val_batches, validation_steps=validation_steps)

    conv_model.optimizer.lr = 1e-7
    conv_model.fit_generator(batches, steps_per_epoch=steps_per_epoch, epochs=10, 
                            validation_data=val_batches, validation_steps=validation_steps)
    conv_model.save_weights(model_path + 'aug' + i + '.h5')

## Build ensemble

In [24]:
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 1s 37us/step - loss: 0.5620 - acc: 0.7707 - val_loss: 0.1551 - val_acc: 0.9425
Epoch 2/12
23000/23000 [==============================] - 1s 36us/step - loss: 0.2222 - acc: 0.9180 - val_loss: 0.1001 - val_acc: 0.9650
Epoch 3/12
23000/23000 [==============================] - 1s 37us/step - loss: 0.1679 - acc: 0.9416 - val_loss: 0.0812 - val_acc: 0.9710
Epoch 4/12
23000/23000 [==============================] - 1s 37us/step - loss: 0.1311 - acc: 0.9546 - val_loss: 0.0729 - val_acc: 0.9755
Epoch 5/12
23000/23000 [==============================] - 1s 37us/step - loss: 0.1156 - acc: 0.9581 - val_loss: 0.0684 - val_acc: 0.9760
Epoch 6/12
23000/23000 [==============================] - 1s 39us/step - loss: 0.1169 - acc: 0.9606 - val_loss: 0.0653 - val_acc: 0.9790
Epoch 7/12
23000/23000 [==============================] - 1s 37us/step - loss: 0.1096 - acc: 0.9644 - val_loss: 0.0635 - val_acc:

Epoch 1/10
719/719 [==============================] - 435s 606ms/step - loss: 0.0414 - acc: 0.9855 - val_loss: 0.0921 - val_acc: 0.9765
Epoch 2/10
719/719 [==============================] - 435s 605ms/step - loss: 0.0523 - acc: 0.9827 - val_loss: 0.0322 - val_acc: 0.9910
Epoch 3/10
719/719 [==============================] - 436s 606ms/step - loss: 0.0459 - acc: 0.9850 - val_loss: 0.0571 - val_acc: 0.9840
Epoch 4/10
719/719 [==============================] - 436s 606ms/step - loss: 0.0355 - acc: 0.9886 - val_loss: 0.0419 - val_acc: 0.9890
Epoch 5/10
719/719 [==============================] - 436s 606ms/step - loss: 0.0342 - acc: 0.9871 - val_loss: 0.0384 - val_acc: 0.9855
Epoch 6/10
719/719 [==============================] - 435s 605ms/step - loss: 0.0374 - acc: 0.9871 - val_loss: 0.0379 - val_acc: 0.9905
Epoch 7/10
719/719 [==============================] - 435s 605ms/step - loss: 0.0284 - acc: 0.9902 - val_loss: 0.0511 - val_acc: 0.9885
Epoch 8/10
719/719 [============================

Epoch 2/2
23000/23000 [==============================] - 27s 1ms/step - loss: 0.0474 - acc: 0.9843 - val_loss: 0.0568 - val_acc: 0.9875
Epoch 1/1
719/719 [==============================] - 218s 303ms/step - loss: 0.0725 - acc: 0.9776 - val_loss: 0.0574 - val_acc: 0.9845
Epoch 1/8
719/719 [==============================] - 440s 612ms/step - loss: 0.0940 - acc: 0.9701 - val_loss: 0.0648 - val_acc: 0.9805
Epoch 2/8
719/719 [==============================] - 436s 606ms/step - loss: 0.0819 - acc: 0.9732 - val_loss: 0.1396 - val_acc: 0.9620
Epoch 3/8
719/719 [==============================] - 435s 605ms/step - loss: 0.0767 - acc: 0.9753 - val_loss: 0.0548 - val_acc: 0.9850
Epoch 4/8
719/719 [==============================] - 435s 605ms/step - loss: 0.0846 - acc: 0.9723 - val_loss: 0.0770 - val_acc: 0.9780
Epoch 5/8
719/719 [==============================] - 435s 605ms/step - loss: 0.0565 - acc: 0.9814 - val_loss: 0.0420 - val_acc: 0.9865
Epoch 6/8
719/719 [==============================] - 4

## Combine ensemble and test

In [25]:
ens_model = vgg_ft_bn(2)
for layer in ens_model.layers: layer.trainable=True

In [26]:
def get_ens_pred(arr, fname):
    ens_pred = []
    for i in range(5):
        i = str(i)
        ens_model.load_weights('{}{}{}.h5'.format(model_path, fname, i))
        preds = ens_model.predict(arr, batch_size=batch_size)
        ens_pred.append(preds)
    return ens_pred

In [27]:
val_pred2 = get_ens_pred(val, 'aug')

In [28]:
val_avg_preds2 = np.stack(val_pred2).mean(axis=0)

In [29]:
categorical_accuracy(val_labels, val_avg_preds2).eval().mean()

0.98949999